In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras import layers
from keras import models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-03-17 08:34:11.136012: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 08:34:11.136106: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 08:34:11.333754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# Define paths to your training and validation data
train_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
validation_dir = '/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

In [4]:
# Data Augmentation for Training Data
train_datagen = ImageDataGenerator(
    rescale=1./255,              # Rescale pixel values to [0, 1]
    rotation_range=40,           # Rotate images randomly up to 40 degrees
    width_shift_range=0.2,       # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2,      # Shift images vertically by up to 20% of the height
    shear_range=0.2,             # Shear transformation with a maximum shear intensity
    zoom_range=0.2,              # Zoom in randomly by up to 20%
    horizontal_flip=True,        # Randomly flip images horizontally
    fill_mode='nearest'          # Fill in missing pixels using the nearest neighbor
)

# Data Augmentation for Validation Data (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # Change to 'categorical' if you have more than 2 classes
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'  # Change to 'categorical' if you have more than 2 classes
)

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [6]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze the pre-trained layers
for layer in vgg_model.layers:
    layer.trainable = False
# Create a new model
model = Sequential([
    vgg_model,
    Flatten(),
#     Dense(256, activation='relu'),
    Dense(38, activation='softmax')  # For binary classification
    # If you have more than two classes, use Dense(num_classes, activation='softmax')
])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# Compile the model
model.compile(
    loss='categorical_crossentropy',  # For binary classification
    # If you have more than two classes, use loss='categorical_crossentropy'
    optimizer=Adam(learning_rate=0.01),     # You can adjust the learning rate as needed
    metrics=['accuracy']
)

In [8]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=1,  # You can adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    verbose = 1
)

/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   1/2196 ━━━━━━━━━━━━━━━━━━━━ 21:15:39 35s/step - accuracy: 0.0000e+00 - loss: 3.8610

I0000 00:00:1710664555.963266      87 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1710664555.983255      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 0s 501ms/step - accuracy: 0.5694 - loss: 10.8645

W0000 00:00:1710665657.929794      87 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


2196/2196 ━━━━━━━━━━━━━━━━━━━━ 1284s 569ms/step - accuracy: 0.5694 - loss: 10.8631 - val_accuracy: 0.8175 - val_loss: 5.2567


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 38)             │       953,382 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,574,836 (67.04 MB)

 Trainable params: 953,382 (3.64 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 1,906,766 (7.27 MB)